https://hwangheek.github.io/2021/black-litterman/ 를 참조하였습니다.

# 1. 블랙-리터만 수식

\begin{align} E[R] &= [(\tau\Sigma)^{-1}+P^T\Omega^{-1}P]^{-1}[(\tau\Sigma)^{-1}\Pi+P^T\Omega^{-1}Q] \\ \hat{w} &= (\Sigma^{-1}E[R])/(1^T\Sigma^{-1}E[R]) \end{align}

[ \begin{align*} N &\qquad\text{is the number of assets} \\ K &\qquad\text{is the number of views} \\ E[R] &\qquad\text{is the new (posterior) Combined Return Vector (N x 1 column vector)} \\ \tau &\qquad\text{is a scalar tuning constant} \\ \Sigma &\qquad\text{is the covariance matrix of excess returns (N x N matrix)} \\ P &\qquad\text{is a matrix that identifies the assets to each views} \\ &\qquad\text{(K x N matrix or 1 x N row vector in the special case of 1 view)} \\ \Omega &\qquad\text{is a diagonal covariance matrix of error terms from the expressed views} \\ &\qquad\text{representing the uncertainty in each view (K x K matrix)} \\ \Pi &\qquad\text{is the Implied Equilibrium Return Vector (N x 1 column vector)} \\ Q &\qquad\text{is the vector of views (K x 1 column vector)} \\ \hat{w} &\qquad\text{is the new portfolio weight (N x 1 column vector) derived by the model} \\ \lambda &\qquad\text{is the risk aversion coefficient} \end{align*}\]

- 시장 데이터를 통해 도출된 수익률과 투자자의 전망(K)을 조합하여 새로운 기대수익률(E[R])을 만드는 과정

-> 도출된 기대수익률을 가지고 새로운 포트폴리오 비중(w_hat)을 구하는 것이 목표

## 2.1 N : The Number of Assets
- 모델에 사용되는 자산의 개수

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import inv # 역행렬 method

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/davidkim0523/Momentum/main/Data.csv', index_col=0)
df

,DAX Index,Eurostoxx 50 Index,FTSE 100 Index,S&P 500 Index,Nasdaq 100 Index,Nikkei 225 Index,Topix Index,KOSPI 200 Index,Hang Seng Index,EURUSD,GBPUSD,SEKUSD,CADUSD,JPYUSD,AUDUSD,NZDUSD,USDKRW,Brent Crude,WTI Crude,Heating Oil,Natural Gas,Gold,Silver,Copper,Corn,Soybean,Eurodollar,UST 2Y,UST 5Y,UST 10Y,UST Long,Euro Schatz,Euro Bobl,Euro Bund,Gilt 10Y,JGB 10Y,AGB 3Y,AGB 10Y
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-03,8768.903220,2443.196385,4663.410938,1110.469607,3472.256961,14612.63447,1183.492979,125.774501,10900.421630,84.7247,98.6827,94.9442,105.6601,106.3850,107.4905,98.5725,92.4692,28.201733,57.313747,76.552150,602.962662,361.488344,7.453264,64.897922,1102.457819,189.890230,83.073442,84.018691,67.205578,56.429022,54.465692,94.644953,80.314443,70.939571,66.326959,101.388587,91.795922,91.597163
2000-01-04,8544.026504,2347.719768,4483.277751,1068.829363,3213.045685,14791.12124,1188.332027,127.696511,10671.302490,85.1545,98.6710,94.8869,105.2203,104.5369,106.9175,97.5191,92.0412,27.425848,57.201806,76.450637,563.351976,563.351976,7.346652,63.769916,1114.814134,193.113996,2136.224480,83.122577,67.431537,67.431537,54.920204,54.920204,94.663425,80.322243,70.829715,65.427993,91.628770,91.439483
2000-01-05,8417.734710,2292.443832,4391.038865,1070.154279,3172.402802,14251.78078,1131.646032,118.414606,9827.014147,85.3527,99.0696,95.0361,105.4202,103.5898,107.3891,97.6676,93.0465,26.683697,55.768962,75.063291,561.280829,561.280829,7.121126,64.409119,1114.814134,195.193845,2157.824323,83.077909,67.173303,67.173303,54.352064,54.352064,94.663425,80.314443,70.644332,65.011255,91.717262,91.557743
2000-01-06,8383.584378,2278.876102,4336.564450,1062.961874,2991.767767,13875.40650,1088.785889,113.633019,9393.884811,85.4018,99.3596,94.9732,104.7016,102.4936,106.6364,96.6142,94.0106,26.560005,55.477916,74.758752,568.529844,568.529844,7.062353,64.145918,1118.932905,194.673883,2147.024402,83.113643,67.291657,67.291657,54.787638,54.787638,94.672662,80.314443,70.548208,64.939814,91.628770,91.459193
2000-01-07,8801.764861,2353.749870,4365.973950,1105.169939,3217.109974,14077.17415,1103.994327,114.383072,9566.508822,85.1203,98.8713,94.4588,104.9930,102.3636,106.8957,97.1570,93.2942,25.964036,54.224178,73.033029,562.575296,562.575296,7.100624,64.221118,1136.780915,196.129777,2175.104198,83.162778,67.442294,67.442294,55.109584,55.109584,94.866625,80.696670,71.124953,65.231531,91.628770,91.478903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-31,15414.000000,4035.000000,7023.500000,4202.400000,13686.500000,28965.00000,1928.500000,427.700000,28958.000000,91.7084,94.8765,90.1362,131.3157,66.2417,193.2133,224.6590,78.1691,69.320000,66.320000,203.830000,2.986000,2.986000,28.014000,467.750000,656.750000,1530.500000,663.500000,99.840000,124.414100,124.414100,158.125000,158.125000,112.045000,134.720000,169.940000,128.270000,99.785000,98.350000
2021-06-01,15567.000000,4070.000000,7078.000000,4198.600000,13648.750000,28760.00000,1920.000000,430.100000,29350.000000,91.6010,94.4625,90.3980,131.2399,66.3015,193.7121,224.2579,77.8423,70.250000,67.720000,207.150000,3.104000,3.104000,28.102000,465.350000,688.750000,1548.500000,693.500000,99.840000,124.367200,124.367200,157.718800,157.718800,112.040000,134.710000,169.830000,127.890000,99.800000,98.370000
2021-06-02,15583.000000,4084.000000,7095.000000,4206.200000,13673.750000,28900.00000,1940.500000,429.500000,29144.000000,91.5835,94.6026,90.4813,131.6326,66.2524,193.7113,223.7024,78.3494,71.350000,68.830000,210.710000,3.075000,3.075000,28.204000,459.150000,675.000000,1562.500000,687.500000,99.840000,124.421900,124.421900,158.093800,158.093800,112.050000,134.820000,170.150000,128.200000,99.800000,98.380000


In [ ]:
index_df = df.iloc[:,[0,1,2,3,4,5,6,7,8]]
index_df

,DAX Index,Eurostoxx 50 Index,FTSE 100 Index,S&P 500 Index,Nasdaq 100 Index,Nikkei 225 Index,Topix Index,KOSPI 200 Index,Hang Seng Index
Date,,,,,,,,,
2000-01-03,8768.903220,2443.196385,4663.410938,1110.469607,3472.256961,14612.63447,1183.492979,125.774501,10900.421630
2000-01-04,8544.026504,2347.719768,4483.277751,1068.829363,3213.045685,14791.12124,1188.332027,127.696511,10671.302490
2000-01-05,8417.734710,2292.443832,4391.038865,1070.154279,3172.402802,14251.78078,1131.646032,118.414606,9827.014147
2000-01-06,8383.584378,2278.876102,4336.564450,1062.961874,2991.767767,13875.40650,1088.785889,113.633019,9393.884811
2000-01-07,8801.764861,2353.749870,4365.973950,1105.169939,3217.109974,14077.17415,1103.994327,114.383072,9566.508822
...,...,...,...,...,...,...,...,...,...
2021-05-31,15414.000000,4035.000000,7023.500000,4202.400000,13686.500000,28965.00000,1928.500000,427.700000,28958.000000
2021-06-01,15567.000000,4070.000000,7078.000000,4198.600000,13648.750000,28760.00000,1920.000000,430.100000,29350.000000
2021-06-02,15583.000000,4084.000000,7095.000000,4206.200000,13673.750000,28900.00000,1940.500000,429.500000,29144.000000


In [ ]:
returns_df = index_df.pct_change()
returns_df

,DAX Index,Eurostoxx 50 Index,FTSE 100 Index,S&P 500 Index,Nasdaq 100 Index,Nikkei 225 Index,Topix Index,KOSPI 200 Index,Hang Seng Index
Date,,,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,-0.025645,-0.039079,-0.038627,-0.037498,-0.074652,0.012215,0.004089,0.015281,-0.021019
2000-01-05,-0.014781,-0.023545,-0.020574,0.001240,-0.012649,-0.036464,-0.047702,-0.072687,-0.079118
2000-01-06,-0.004057,-0.005918,-0.012406,-0.006721,-0.056940,-0.026409,-0.037874,-0.040380,-0.044075
2000-01-07,0.049881,0.032856,0.006782,0.039708,0.075321,0.014541,0.013968,0.006601,0.018376
...,...,...,...,...,...,...,...,...,...
2021-05-31,-0.006318,-0.007624,0.000000,0.000000,0.000000,-0.004810,-0.010010,0.003166,-0.000966
2021-06-01,0.009926,0.008674,0.007760,-0.000904,-0.002758,-0.007078,-0.004408,0.005611,0.013537
2021-06-02,0.001028,0.003440,0.002402,0.001810,0.001832,0.004868,0.010677,-0.001395,-0.007019


In [ ]:
N = returns_df.columns.size
N

9

### Equal Weights

In [ ]:
weights = {'DAX Index' : 1/9, 
           'Eurostoxx 50 Index' : 1/9, 
           'FTSE 100 Index' : 1/9, 
           'S&P 500 Index' : 1/9,
           'Nasdaq 100 Index' : 1/9, 
           'Nikkei 225 Index' : 1/9, 
           'Topix Index' : 1/9,
           'KOSPI 200 Index' : 1/9, 
           'Hang Seng Index' : 1/9}
mkt_W = pd.Series(weights)
mkt_W

DAX Index             0.111111
Eurostoxx 50 Index    0.111111
FTSE 100 Index        0.111111
S&P 500 Index         0.111111
Nasdaq 100 Index      0.111111
Nikkei 225 Index      0.111111
Topix Index           0.111111
KOSPI 200 Index       0.111111
Hang Seng Index       0.111111
dtype: float64

### to monthly returns

In [ ]:
returns_df = returns_df.reset_index()
returns_df['Date'] = pd.to_datetime(returns_df['Date'])
monthly_returns_df = returns_df.set_index('Date').resample('M').last()
monthly_returns_df

,DAX Index,Eurostoxx 50 Index,FTSE 100 Index,S&P 500 Index,Nasdaq 100 Index,Nikkei 225 Index,Topix Index,KOSPI 200 Index,Hang Seng Index
Date,,,,,,,,,
2000-01-31,-0.033404,-0.015956,-0.016047,0.025247,0.039568,0.005679,0.002053,0.003756,-0.040000
2000-02-29,0.012912,0.016585,0.027285,0.017993,0.034392,0.009346,-0.002884,0.006314,0.019185
2000-03-31,-0.012081,0.000574,0.016791,0.007313,0.030040,-0.001957,-0.002049,-0.026549,-0.008799
2000-04-30,0.033380,0.029281,0.028431,-0.004941,0.033921,0.002212,0.005765,0.037500,0.025828
2000-05-31,0.000140,0.008136,-0.000314,-0.003503,-0.029764,0.002760,0.001315,0.056102,0.047904
...,...,...,...,...,...,...,...,...,...
2021-02-28,-0.009716,-0.016540,-0.028367,-0.004911,0.006176,-0.031126,-0.028993,-0.030736,-0.033411
2021-03-31,-0.000333,-0.001550,-0.007725,0.004990,0.016423,-0.008147,-0.012133,-0.002166,-0.007327
2021-04-30,0.000396,-0.004044,0.000866,-0.006899,-0.007417,-0.005502,-0.002361,-0.007757,-0.019823


## 2.2 Sigma : Covariance Matrix of Asset Excess Returns
- 각 자산별 수익률의 공분산행렬


In [ ]:
sigma = monthly_returns_df.cov()
sigma

,DAX Index,Eurostoxx 50 Index,FTSE 100 Index,S&P 500 Index,Nasdaq 100 Index,Nikkei 225 Index,Topix Index,KOSPI 200 Index,Hang Seng Index
DAX Index,0.000163,0.000145,0.000097,0.000060,0.000068,0.000045,0.000043,0.000072,0.000061
Eurostoxx 50 Index,0.000145,0.000151,0.000103,0.000063,0.000075,0.000042,0.000042,0.000070,0.000062
FTSE 100 Index,0.000097,0.000103,0.000125,0.000053,0.000061,0.000037,0.000036,0.000049,0.000062
S&P 500 Index,0.000060,0.000063,0.000053,0.000095,0.000110,0.000018,0.000015,0.000014,0.000023
Nasdaq 100 Index,0.000068,0.000075,0.000061,0.000110,0.000199,0.000033,0.000025,0.000022,0.000020
Nikkei 225 Index,0.000045,0.000042,0.000037,0.000018,0.000033,0.000183,0.000176,0.000081,0.000098
Topix Index,0.000043,0.000042,0.000036,0.000015,0.000025,0.000176,0.000190,0.000076,0.000097
KOSPI 200 Index,0.000072,0.000070,0.000049,0.000014,0.000022,0.000081,0.000076,0.000185,0.000096
Hang Seng Index,0.000061,0.000062,0.000062,0.000023,0.000020,0.000098,0.000097,0.000096,0.000169


## 2.3 Reverse Optimization Process
- 시장에서 관측되는 자산비중을 기반으로 균형 수익률을 찾아내는 과정

### Mean-Variance Optimization process
- 자산의 수익률과 공분산 데이터를 사용하여 "최대 수익률, 최저 변동성"을 가지는 포트폴리오 비중 구하는 과정

아래는 최적화 과정


\[\begin{align} & \underset{w}{\operatorname{max}}\left(w^T\mu - \frac{1}{2}\lambda w^T\Sigma w\right) \\ & \rightarrow \mu - \lambda\Sigma w = 0 \\ & \rightarrow w = (\lambda\Sigma)^{-1}\mu \end{align}\]

자산 간 공분산행렬이 있는 경우엔 위에 w = 식을 아래와 같이 변형하여 각 자산의 implied returns를 역으로 구할 수 있다.

\[\begin{align} \mu_{implied} = \lambda \Sigma w_{mkt} \end{align}\]

## 2.4. lambda : Risk Aversion Coefficient
- lambda가 클수록, implied returns인 pi 값이 커진다.


### 값을 설정하는 방법
-  시가총액가중 포트폴리오의 평균 초과수익률 / 분산


In [ ]:
expected_portfolio_return = monthly_returns_df.mean().multiply(mkt_W).sum()
portfol_var = mkt_W.dot(sigma).dot(mkt_W)
lmbda = expected_portfolio_return / portfol_var
lmbda 

-1.6623144789154247

## 2.5. Pi : Implied Excess Equilibrium Return Vector

\[\begin{equation} \Pi = \lambda \Sigma w_{mkt} \end{equation}\]

\[\begin{align*} \lambda &\qquad\text{is the risk aversion coefficient} \\ \Sigma &\qquad\text{is the covariance matrix of excess returns (N x N matrix)} \\ w_{mkt} &\qquad\text{is the market capitalization weight (N x 1 column vector) of the assets} \end{align*}\]

In [ ]:
Pi = lmbda * sigma.dot(mkt_W)
Pi

DAX Index            -0.000139
Eurostoxx 50 Index   -0.000139
FTSE 100 Index       -0.000115
S&P 500 Index        -0.000083
Nasdaq 100 Index     -0.000113
Nikkei 225 Index     -0.000132
Topix Index          -0.000129
KOSPI 200 Index      -0.000123
Hang Seng Index      -0.000127
dtype: float64

## 2.6. K : The Number of Views
- absolute or relative views on returns

In [ ]:
K = 3

## 2.7. Q : View Vector

각 view들의 예상 수치(예상 절대/상대수익률)를 포함하는, \(K\)x\(1\) column matrix

\[\begin{equation*} K = \begin{bmatrix} 0.02 \\ 0.03 \\ 0.01 \\ \end{bmatrix} \end{equation*}\]

In [ ]:
Q = np.array([0.02, 0.03, 0.01])
assert Q.shape == (K,)

## 2.8. P : Picking Matrix
View에 포함되는 자산들을 명시해주는 행렬

총 3개의 view와 9개의 자산이 있으므로, P는 3x9 행렬

\[\begin{equation*} P = \begin{bmatrix} 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 \\ 1 & 0 & -0.5 & -0.5 & 0 & 0 & 0 & 0 & 0 \\ \end{bmatrix} \end{equation*}\]

첫 번째 행은 Absolute view, 하나의 column만 1이 되는 row 형태

두 번째와 세 번째 행은 outperform 할 것으로 예상되는 자산은 양수, underperform 할 것으로 예상되는 자산은 음수로 표시. 
row의 총 합이 0이 되도록 맞춤 

세 번째 행처럼 다수의 자산을 명시할 경우,예상 outperform 자산의 합은 1, 예상 underperform 자산의 합은 -1, row 총 합은 0이 되도록 분배 

In [ ]:
P = np.array([
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, -1],
    [1, 0, -.5, -.5, 0, 0, 0, 0, 0]
])
assert P.shape == (K, N)

## 2.9. tau : Scalar "Tuning" Constant
= Prior estimate of returns (Pi)에 대한 불확실 정도를 나타내는 수치

- tau 값이 작을수록, 모델에 의해 계산되는 combined return vector(\(E[R]\))가 implied equilibrium return(Pi))에 가까워지게 됨

- tau 값을 구하는 일관된 기준이 없어서 omega 값에 tau 값을 factor-out 시키도록 값을 설정하는 방법 있음


In [ ]:
tau = 1 # 0이 아닌 임의의 값

## 2.10. Omega : Uncertainty Matrix of Views
= 개별 view들의 불확실 정도를 나타내는 대각행렬

- 행렬의 원소인 omega_k가 0에 가까울수록 view k의 confidence가 100%에 가깝다는 것을 의미, omega_k가 클수록 해당 view의 불확실성이 높음을 의미

\[\begin{equation} \Omega = \begin{bmatrix} \omega_1 & 0 & 0 \\ 0 & \ddots & 0 \\ 0 & 0 & \omega_k \\ \end{bmatrix} \end{equation}\]

\[\begin{equation*} \Omega = diag(P(\tau\Sigma)P^T) = \begin{bmatrix} (p_1 \Sigma p_1^T)*\tau & 0 & 0 \\ 0 & \ddots & 0 \\ 0 & 0 & (p_K \Sigma p_K^T)*\tau \\ \end{bmatrix} \end{equation*}\]

위와 같이 Omega 설정하면 tau 값이 변해도 E[R]값이 변하지 않아서 view와 관련된 input만 신경쓰면 됨

In [ ]:
Omega = tau*P.dot(sigma).dot(P.T) * np.eye(K)
Omega

array([[ 1.90287895e-04, -0.00000000e+00,  0.00000000e+00],
       [-0.00000000e+00,  1.62787747e-04,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  8.76791375e-05]])

# 3. Data ready. Run it (no confidence level of views)

In [ ]:
ER = Pi + tau*sigma.dot(P.T).dot(inv(P.dot(tau*sigma).dot(P.T) + Omega).dot(Q - P.dot(Pi)))

In [ ]:
w_hat = inv(sigma).dot(ER)
w_hat = pd.Series(w_hat / w_hat.sum(), index=ER.index)
w_hat

DAX Index             0.724180
Eurostoxx 50 Index   -0.003382
FTSE 100 Index       -0.367163
S&P 500 Index        -0.367163
Nasdaq 100 Index     -0.003382
Nikkei 225 Index     -0.003382
Topix Index           1.027056
KOSPI 200 Index       1.700055
Hang Seng Index      -1.706819
dtype: float64

In [ ]:
w_hat - mkt_W

DAX Index             0.613069
Eurostoxx 50 Index   -0.114493
FTSE 100 Index       -0.478274
S&P 500 Index        -0.478274
Nasdaq 100 Index     -0.114493
Nikkei 225 Index     -0.114493
Topix Index           0.915945
KOSPI 200 Index       1.588944
Hang Seng Index      -1.817930
dtype: float64

# 4. Considering Confidence Level of Views

## 각 view의 confidence level 값을 모델에 적용하는 방법





### 1. 새로 계산될 Omega의 결과를 담아둘 행렬 선언

In [ ]:
from scipy import optimize

In [ ]:
implied_Omega = np.zeros((k,k))

### 2. 각 view의 confidence level들로 이루어진 행렬 C 정의
\[\begin{equation} C = \begin{bmatrix} 0.75 \\ 0.25 \\ 0.50 \\ \end{bmatrix} \end{equation}\]

In [ ]:
C = np.array([0.75, 0.25, 0.50])
assert C.shape == (K,)

### 3. View k가 모델의 유일한 view이고 100% 확실다는 가정하에, E[R] 값과 w 값을 계산

View k가 100% 확실한 경우, (1)~(2)의 수식은 아래와 같이 정리할 수 있다. \[\begin{align} E[R_{\text{k,100%}}] &= \Pi + \tau\Sigma P_{k}^T (P_k \tau\Sigma P_{k}^T)^{-1}\\ w_{\text{k,100%}} &= (\Sigma^{-1}E[R_{\text{k,100%}}])/(1^T\Sigma^{-1}E[R_{\text{k,100%}}]) \end{align}\]

In [ ]:
# 기존 ER을 구하는 수식에서 P를 P[None, k]로, Q를 Q[None, k]로 대체하고, Omega를 생략한 수식
ER_k_100 = Pi + tau*sigma.dot(P[None, k].T).dot(inv(P[None, k].dot(tau * sigma).dot(P[None, k].T)).dot(Q[None, k] - P[None, k].dot(Pi)))
w_k_100 = inv(sigma).dot(ER_k_100)
w_k_100 = pd.Series(w_k_100 / w_k_100.sum(), index=ER.index)
w_k_100

DAX Index            -0.001774
Eurostoxx 50 Index   -0.001774
FTSE 100 Index       -0.001774
S&P 500 Index        -0.001774
Nasdaq 100 Index     -0.001774
Nikkei 225 Index     -0.001774
Topix Index           1.014191
KOSPI 200 Index      -0.001774
Hang Seng Index      -0.001774
dtype: float64

### 4. View k가 모델의 유일한 view라는 가정하에, 주어진 confidence level을 반영되었을 경우의 w 를 추론

View k의 confidence level이 100%인 경우 포트폴리오 비중이 \(w_{\text{k,100%}}\)이고, confidence level이 0%인 경우 포트폴리오 비중이 \(w_{mkt}\)이다. 따라서, confidence level이 \(C_k\)인 경우 포트폴리오 비중을 다음과 같이 추론할 수 있다.

\[\begin{align} w_{\text{k,%}} = w_{mkt} + (w_{\text{k,100%}} - w_{mkt}) * C_k \end{align}\]


In [ ]:
w_k_implied = mkt_W + (w_k_100 - mkt_W) * C[k]

### 5. View k가 모델의 유일한 view라는 가정하에, view k의 confidence interval 값을 조절해보면서, confidence interval이 반영된 포트폴리오의 결과가 w_k,%에 가깝도록 하는 omega_k 값을 찾는다.

In [ ]:
def func(omega_k):
    # 기존 ER을 구하는 수식에서 P를 P[None, k]로, Q를 Q[None, k]로 대체한 수식
    ER_k = Pi + tau * sigma.dot(P[None, k].T).dot(inv(P[None, k].dot(tau * sigma).dot(P[None, k].T) + omega_k).dot(Q[None, k] - P[None, k].dot(Pi)))
    
    w_k = inv(sigma).dot(ER_k)
    w_k = pd.Series(w_k / w_k.sum(), index=ER.index)
    
    diff = w_k_implied - w_k
    return diff.T.dot(diff)

implied_Omega[k][k] = optimize.minimize_scalar(
    fun=func,
    bounds=(1e-8, 1e+12),
    method='bounded',
).x

IndexError: ignored

### 6. 위 3~5번 과정을 모든 k에 대해 iterate하여 implied_Omega를 구함

In [ ]:
implied_Omega

array([], shape=(0, 0), dtype=float64)

In [ ]:
ER_with_CL = Pi + tau * sigma.dot(P.T).dot(inv(P.dot(tau * sigma).dot(P.T) + implied_Omega).dot(Q - P.dot(Pi)))
w_hat_with_CL = inv(sigma).dot(ER_with_CL)
w_hat_with_CL = pd.Series(w_hat_with_CL / w_hat_with_CL.sum(), index=ER.index)
w_hat_with_CL

ValueError: ignored

# 5. Dealing with Constraints